# Bienvenidos a :snowflake: Notebooks de Snowflake :notebook:

¡Lleva tu análisis de datos al siguiente nivel trabajando con Python y SQL sin problemas en [Snowflake Notebooks](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about)! ⚡️

¡Aquí tienes un cuaderno rápido para comenzar con tu primer proyecto! 🚀

## Añadiendo Paquetes de Python 🎒

Los Notebooks vienen preinstalados con bibliotecas comunes de Python para ciencia de datos 🧪 y aprendizaje automático 🧠, como numpy, pandas, matplotlib, ¡y más!

Si deseas utilizar otros paquetes, haz clic en el menú desplegable `Packages` en la parte superior derecha para agregar paquetes adicionales a tu notebook.

Para el propósito de esta demostración, los paquetes `matplotlib` y `scipy` se agregaron como parte de environment.yml al crear el Notebook.

In [ ]:
# Import Python packages used in this notebook
import streamlit as st
import altair as alt

# Pre-installed libraries that comes with the notebook
import pandas as pd
import numpy as np

# Package that we just added
import matplotlib.pyplot as plt

## Consulta SQL al alcance de tu mano 💡

Podemos cambiar fácilmente entre Python y SQL en la misma hoja de trabajo.

Escribamos algo de SQL para generar datos de muestra con los que jugar.

In [ ]:
-- Generating a synthetic dataset of Snowboard products, along with their price and rating
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL(5, 3, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

## De Vuelta a Trabajar en Python 🐍

Puedes darle un nombre a las celdas y referirte a su salida en celdas posteriores.

Podemos acceder a los resultados de SQL directamente en Python y convertir los resultados en un dataframe de pandas. 🐼

```python
# Accede a la salida de la celda SQL como un dataframe de Snowpark
my_snowpark_df = sql_querying.to_df()
```

```python
# Convierte la salida de una celda SQL en un dataframe de pandas
my_df = sql_querying.to_pandas()
```

In [ ]:
df = sql_querying.to_pandas()
df

## 📊 Visualiza tus datos

Podemos usar [Altair](https://altair-viz.github.io/) para visualizar fácilmente la distribución de nuestros datos como un histograma.

In [ ]:
# Let's plot the results with Altair
chart = alt.Chart(df,title="Rating Distribution").mark_bar().encode(
    alt.X("RATING", bin=alt.Bin(step=2)),
    y='count()',
)

st.altair_chart(chart)

Digamos que deseas personalizar tu gráfico y trazar la estimación de densidad del kernel (KDE) y la mediana. Podemos usar matplotlib para trazar la distribución de precios. Ten en cuenta que el comando `.plot` usa `scipy` internamente para calcular el perfil KDE, que agregamos como paquete anteriormente en este tutorial.

In [ ]:
fig, ax = plt.subplots(figsize = (6,3))
plt.tick_params(left = False, right = False , labelleft = False) 

price = df["PRICE"]
price.plot(kind = "hist", density = True, bins = 15)
price.plot(kind="kde", color='#c44e52')


# Calculate percentiles
median = price.median()
ax.axvline(median,0, color='#dd8452', ls='--')
ax.text(median,0.8, f'Median: {median:.2f}  ',
        ha='right', va='center', color='#dd8452', transform=ax.get_xaxis_transform())

# Make our chart pretty
plt.style.use("bmh")
plt.title("Price Distribution")
plt.xlabel("PRICE (binned)")
left, right = plt.xlim()   
plt.xlim((0, right))  
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.show()

## Trabajando con datos usando Snowpark 🛠️

Además de usar tus bibliotecas de ciencia de datos de Python favoritas, también puedes usar la [API de Snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/index) para consultar y procesar tus datos a escala dentro del Notebook.

Primero, puedes obtener tu variable de sesión directamente a través de la sesión activa del notebook. La variable de sesión es el punto de entrada que te da acceso al uso de la API de Python de Snowflake.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Por ejemplo, podemos usar Snowpark para guardar nuestro dataframe de pandas de nuevo en una tabla en Snowflake. 💾

In [ ]:
session.write_pandas(df,"SNOW_CATALOG",auto_create_table=True, table_type="temp")

Ahora que la tabla `SNOW_CATALOG` ha sido creada, podemos cargar la tabla usando la siguiente sintaxis:

```python
df = session.table("<DATABASE_NAME>.<SCHEMA_NAME>.<TABLE_NAME>")
```

Si tu sesión ya está configurada en la base de datos y el esquema para la tabla a la que deseas acceder, entonces puedes hacer referencia al nombre de la tabla directamente.

In [ ]:
df = session.table("SNOW_CATALOG")

Una vez que hemos cargado la tabla, podemos llamar a [`describe`](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.describe) de Snowpark para calcular estadísticas descriptivas básicas.

In [ ]:
df.describe()

## Usando variables de Python en celdas SQL 🔖

Puedes usar la sintaxis Jinja `{{..}}` para referirte a variables de Python dentro de tus consultas SQL de la siguiente manera.

```python
threshold = 5
```

```
-- Referencia a la variable de Python en SQL
SELECT * FROM SNOW_CATALOG where RATING > {{threshold}}
```
Del mismo modo, puedes hacer referencia a un dataframe de Pandas dentro de tu declaración SQL:
```
-- Filtrando desde un dataframe de Pandas
SELECT * FROM {{my_df}} where VAR = 6
```

Pongamos esto en práctica para generar una distribución de valores para las clasificaciones basadas en los valores de media y desviación estándar que establecimos con Python.

In [ ]:
mean = 5 
stdev = 3

In [ ]:
-- Note how we use the Python variables `mean` and `stdev` to populate the SQL query
-- Note how the Python variables dynamically populate the SQL query
CREATE OR REPLACE TABLE SNOW_CATALOG AS 
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL({{mean}}, {{stdev}}, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

In [ ]:
SELECT * FROM SNOW_CATALOG;

### ¡Sube de nivel tu juego de subconsultas! 🧑‍🎓

Puedes simplificar subconsultas largas con [CTEs](https://docs.snowflake.com/en/user-guide/queries-cte) combinando lo que hemos aprendido con la referencia de resultados de celdas de Python y SQL.

Por ejemplo, si queremos calcular la calificación promedio de todos los productos con calificaciones superiores a 5. Normalmente tendríamos que escribir algo como lo siguiente:


In [ ]:
WITH RatingsAboveFive AS (
    SELECT RATING
    FROM SNOW_CATALOG
    WHERE RATING > 5
)
SELECT AVG(RATING) AS AVG_RATING_ABOVE_FIVE
FROM RatingsAboveFive;

Con Snowflake Notebooks, ¡la consulta es mucho más simple! Puedes obtener el mismo resultado filtrando una tabla SQL desde otra celda SQL haciendo referencia a ella con Jinja, p.ej., `{{my_cell}}`.

In [ ]:
SELECT AVG(RATING) FROM {{variables3}}
WHERE RATING > 5

## Creando una aplicación interactiva con Streamlit 🪄

Juntando todo esto, construyamos una aplicación Streamlit para explorar cómo diferentes parámetros impactan la forma del histograma de distribución de datos.

In [ ]:
import streamlit as st
st.markdown("# Mueve los filtros y observa cómo se actualizan los resultados.! 👇")
col1, col2 = st.columns(2)
with col1:
    mean = st.slider('Media de distribución de las calificaciones',0,10,3) 
with col2:
    stdev = st.slider('Desviacion estandar de distribución de las calificaciones', 0, 10, 5)

query =f'''CREATE OR REPLACE TABLE SNOW_CATALOG AS 
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL({mean}, {stdev}, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));'''
session.sql(query).collect()


# Read table from Snowpark and plot the results
df = session.table("SNOW_CATALOG").to_pandas()
# Let's plot the results with Altair
alt.Chart(df).mark_bar().encode(
    alt.X("RATING", bin=alt.Bin(step=2)),
    y='count()',
)

## Puedes ser más rápido con atajos de teclado 🏃

Estos atajos pueden ayudarte a navegar por tu notebook más rápidamente.

| Comando                     | Atajo            |
| --------------------------- | ---------------- |
| **Ejecutar esta celda y avanzar** | SHIFT + ENTER    |
| **Ejecutar solo esta celda**   | CMD + ENTER      |
| **Ejecutar todas las celdas** | CMD + SHIFT + ENTER |
| **Agregar celda ABAJO**     | b                |
| **Agregar celda ARRIBA**    | a                |
| **Eliminar esta celda**     | d+d              |

\
Puedes ver la lista completa de atajos haciendo clic en el botón `?` en la parte inferior derecha.

In [ ]:
-- Teardown code to cleanup environment after tutorial
DROP TABLE SNOW_CATALOG;

## ¡Sigue explorando Notebooks! 🧭

¡Consulta nuestra [galería de notebooks de ejemplo](https://github.com/Snowflake-Labs/notebook-demo) y [documentación](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about) para obtener más información!